# Week 3

In [47]:
import pandas as pd
import geocoder

## 1. Setting Variables

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

## 2. Using pd.read_html with header = 0.
This returns a list

In [34]:
html_list = pd.read_html(url, header = 0)
html_list

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 282      M8Z         Etobicoke              Mimico NW
 283      M8Z         Etobicoke     The Queensway West
 284      M8Z         Etobicoke  Royal York South West
 285      M8Z         Etobicoke         South of Bloor
 286      M9Z      Not assigned           Not assigned
 
 [287 rows x 3 columns],
                                           Unnamed: 0  \
 0  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
 1                                                 NL   
 2                                                  A   
 
                                Canadian postal codes  \
 0  NL NS PE NB QC ON MB S

## 3. Data is located at index 0. Setting data variable to the value located at index 0

In [36]:
data = html_list[0]
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 4. Removing all entries that contain "Not Assigned"

In [39]:
data_boroughs_cleaned = data[data.Borough != 'Not assigned']

In [41]:
data_boroughs_cleaned.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


## 5. Renaming all neighborhoods with name "Not Assigned" to the borough name.

In [43]:
data_neighborhood_renamed = data_boroughs_cleaned.copy()
data_neighborhood_renamed.Neighbourhood[data_neighborhood_renamed.Neighbourhood == 'Not assigned'] = data_neighborhood_renamed.Borough[data_neighborhood_renamed.Neighbourhood == 'Not assigned']

In [45]:
data_neighborhood_renamed.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North
